In [1]:
import os
import gc
import numpy as np
import tomopy
import skimage
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import torch.nn.functional as F
import random
import torch
import scipy.io as scio
import astra
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import time
from tomocupy_stream import GPURecRAM
from tomocupy_stream import find_center
import tifffile
import dxchange
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
DEVICE="2"

/home/cxhpc/anaconda3/envs/tomocupy/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')
astropy module not found


In [2]:
alignment_data=np.load("./Data/XANES_00089/alignment_result/alignment_8570.npy")
print(alignment_data.shape)

(180, 331, 402)


In [3]:
proj=alignment_data
c,h,w=proj.shape
dark = np.zeros([1,h,w],dtype='float32')
flat = np.ones([1,h,w],dtype='float32')
data = np.zeros([c,h,w],dtype='float32')
data = data.swapaxes(0,1)
theta = np.linspace(0,180,180).astype('float32')
center_search_width = 1
center_search_step = 0.1
center_search_ind = data.shape[0]//2
tmp=proj.swapaxes(0,1)
print(tmp.shape)
rotation_axis = find_center.find_center_vo(tmp, dark, flat,
                                           ind=center_search_ind,
                                           smin=-center_search_width, 
                                           smax=center_search_width, 
                                           step=center_search_step)
print('auto rotation axis',rotation_axis)

cl = GPURecRAM.for_data_like(
    data=data,
    dark=dark,
    flat=flat,
    ncz=8,  # chunk size for GPU processing (multiple of 2), 
    rotation_axis=rotation_axis,  # rotation center
    dtype="float32",  # computation type, note  for float16 n should be a power of 2
    reconstruction_algorithm='fourierrec',
    fbp_filter='parzen', 
    minus_log=False)
obj_proj = cl.recon_all(tmp, dark, flat, theta)
# obj_proj=tomopy.circ_mask(obj_proj,axis=0,ratio=0.8)
print(obj_proj.shape)
print(np.max(obj_proj), np.min(obj_proj))
re_proj = cl.proj_all(obj_proj,theta)
re_proj=re_proj.swapaxes(0,1)
print(re_proj.shape)
print(np.max(re_proj),np.min(re_proj))

(331, 180, 402)
auto rotation axis 205.99999999999994
chunk 042/042 |  |████████████████████████████████████████| 100.0% 
(331, 402, 402)
0.0031520317 -0.00896219
chunk 042/042 |  |████████████████████████████████████████| 100.0% 
(180, 331, 402)
21.085974 -0.5108855


In [4]:
print(obj_proj.shape)
obj_proj=np.abs(obj_proj)
def plot_projections(sid=1):
    plt.imshow(obj_proj[sid,:,:],cmap='gray')
interact(plot_projections, sid = widgets.IntSlider(value=obj_proj.shape[0]//2,
                                               min=0,
                                               max=obj_proj.shape[0]-1,
                                               step=1))

(331, 402, 402)


interactive(children=(IntSlider(value=165, description='sid', max=330), Output()), _dom_classes=('widget-inter…

<function __main__.plot_projections(sid=1)>

In [26]:
energy=8570
obj_proj=obj_proj/np.max(obj_proj)
for idx in range(1, obj_proj.shape[0] + 1):
    slice = obj_proj[idx-1, :, :]
    slice = (slice * 255).astype(np.uint8)
    filename = f"./XANES_00089/test_recon/{idx:06}_{energy:05d}.00_eV_{idx:05d}.tif"
    tifffile.imwrite(filename, slice)

In [5]:
def plot_projections(sid=1):
    plt.imshow(alignment_data[sid,:,:],cmap='gray')
interact(plot_projections, sid = widgets.IntSlider(value=alignment_data.shape[0]//2,
                                               min=0,
                                               max=alignment_data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=90, description='sid', max=179), Output()), _dom_classes=('widget-intera…

<function __main__.plot_projections(sid=1)>